# SPARQL performance comparison
Using `rdflib` for SPARQL queries is handy, but it may not be the most efficient. It might be more efficient to load the RDF data on a dedicated triplestore (such as Virtuoso) and then run SPARQL queries against this repository.

In this notebook we make a small performance test, showing the efficiency gained by using this approach. Before running this notebook, follow the [Notes on loading RadLex in virtuoso](Notes_on_loading_RadLex_in_virtuoso.md).

## Preparation
We start by preparing the connection to the rdflib graph and triplestore respectively.

In [1]:
from rdflib import Graph, ConjunctiveGraph
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [2]:
g = ConjunctiveGraph()
g.parse("../../Data/owlapi.xrdf");

In [3]:
from SPARQLWrapper import SPARQLWrapper, CSV
from io import StringIO

In [4]:
sparql = SPARQLWrapper("http://localhost:8890/sparql")

## Query
We will use the same SPARQL query with both rdflib graph and triplestore. Because both of these can be queried via SPARQL, no changes are required.

In [5]:
qry = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX radlex: <http://www.radlex.org/RID/#>

SELECT DISTINCT ?RID ?name ?synonym ?sub_name1 ?sub1_RID ?synonym1
WHERE
{
 ?term radlex:Preferred_name ?name .
 FILTER regex(?name,"cancer",'i')
 ?term rdfs:label ?RID .
 OPTIONAL{?term radlex:Synonym ?synonym  .}
 
 OPTIONAL{?term radlex:Has_Subtype ?Sub_type1  .
 ?Sub_type1 radlex:Preferred_name ?sub_name1 .
 ?Sub_type1 rdfs:label ?sub1_RID .
 
 OPTIONAL{?Sub_type1 radlex:Synonym ?synonym1  .}
 
 OPTIONAL{?Sub_type1 radlex:Has_Subtype ?Sub_type2  .
 ?Sub_type2 radlex:Preferred_name ?sub_name2 .
 ?Sub_type2 rdfs:label ?sub2_RID .
 
 OPTIONAL{?Sub_type2 radlex:Synonym ?synonym2  .}
 
 OPTIONAL{?Sub_type2 radlex:Has_Subtype ?Sub_type3  .
 ?Sub_type3 radlex:Preferred_name ?sub_name3 .
 ?Sub_type3 rdfs:label ?sub3_RID .
 
 OPTIONAL{?Sub_type3 radlex:Synonym ?synonym3  .}
 

 }}}}
 """

## Timing queries
Now we use the `timeit` magic command to check how long it takes to query the rdflib graph and triple store and generate a pandas DataFrame from the results.

As it can be seen below, using a triplestore is much faster!

In [6]:
%%timeit
rows  = g.query(qry)
res = [{str(k): str(v) for k, v in binding.items()} for binding in rows.bindings]
df_rdflib = pd.DataFrame(res)

1min 58s ± 726 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
sparql.setQuery(qry)
sparql.setReturnFormat(CSV)
res = sparql.queryAndConvert()
resAsStr = res.decode('utf-8')
df_wrapper = pd.read_csv(StringIO(resAsStr))

62.9 ms ± 767 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Comparing results
To make sure we get the same results, we compare the data from both processes.

In [9]:
# We need to generate pandas dataframes again because `timeit` removes all results from workspace.
rows  = g.query(qry)
res = [{str(k): str(v) for k, v in binding.items()} for binding in rows.bindings]
df_rdflib = pd.DataFrame(res)

sparql.setQuery(qry)
sparql.setReturnFormat(CSV)
res = sparql.queryAndConvert()
resAsStr = res.decode('utf-8')
df_wrapper = pd.read_csv(StringIO(resAsStr))

### Visual inspection
First we perfomr a visual inspections from the results. As it can be seen below, it looks like both processes are generating the same results.

In [10]:
df_rdflib.head(200)

,RID,name,sub1_RID,sub_name1,synonym,synonym1
0,RID45697,head and neck cancer,NaN,NaN,NaN,NaN
1,RID45683,cervical cancer,NaN,NaN,NaN,NaN
2,RID34623,gastric cancer,NaN,NaN,stomach cancer,NaN
3,RID45696,esophageal cancer,NaN,NaN,NaN,NaN
4,RID45686,lung cancer,NaN,NaN,NaN,NaN
5,RID45691,thyroid cancer,NaN,NaN,NaN,NaN
6,RID45688,pancreatic cancer,NaN,NaN,NaN,NaN
7,RID45689,prostate cancer,RID49502,clinically significant prostate cancer,cancer of prostate,intermediate high risk prostate cancer
8,RID45689,prostate cancer,RID49502,clinically significant prostate cancer,carcinoma of prostate,intermediate high risk prostate cancer
9,RID45689,prostate cancer,RID49502,clinically significant prostate cancer,"prostate cancer, NOS",intermediate high risk prostate cancer


In [11]:
df_wrapper.head(200)

,RID,name,synonym,sub_name1,sub1_RID,synonym1
0,RID45683,cervical cancer,NaN,NaN,NaN,NaN
1,RID45688,pancreatic cancer,NaN,NaN,NaN,NaN
2,RID45696,esophageal cancer,NaN,NaN,NaN,NaN
3,RID34623,gastric cancer,stomach cancer,NaN,NaN,NaN
4,RID45689,prostate cancer,cancer of prostate,clinically significant prostate cancer,RID49502,intermediate high risk prostate cancer
5,RID45689,prostate cancer,carcinoma of prostate,clinically significant prostate cancer,RID49502,intermediate high risk prostate cancer
6,RID45689,prostate cancer,"prostate cancer, NOS",clinically significant prostate cancer,RID49502,intermediate high risk prostate cancer
7,RID45682,breast cancer,NaN,NaN,NaN,NaN
8,RID45684,colorectal cancer,NaN,NaN,NaN,NaN
9,RID45685,endometrial cancer,NaN,NaN,NaN,NaN


### In depth look
Now we analyze the data which is returned by both queries. Since both queries generate the same list of cancer values (same number of values and the same values), we can be assured that both queries have the same effect.

In [12]:
def listOfTerms(df):
    Lst = [df['name'].tolist(), df['sub_name1'].tolist(), df['synonym'].tolist(), df['synonym1'].tolist()]
    return Lst

In [13]:
def cancerValues(df):
    Cancer_values = []
    Lst = listOfTerms(df)
    for lists in Lst:
        for string in lists:
            if str(string) == 'nan':
                pass
            else:
                Cancer_values.append(string)

    Cancer_values = list(set(Cancer_values))
    return Cancer_values

In [15]:
cv_rdf = cancerValues(df_rdflib)
cv_wrapper = cancerValues(df_wrapper)

assert len(cv_rdf)==len(cv_wrapper) # Same number of values are produced
assert set(cv_rdf)==set(cv_wrapper) # Same values are produced